In [4]:
import pandas as pd

fiber_df = pd.read_excel("clustered_fiber_composition.xlsx")

trousers = fiber_df.dropna(subset=["Category"])
trousers = trousers[trousers["Category"].str.contains("trousers", case=False)]

selected = trousers.iloc[0]

In [5]:

fiber_types = ["Cotton", "Polyester", "Acrylic", "Wool", "Viscose", "Silk", "Polyamide/nylon"]


demand_dict = {
    fiber: selected[fiber] / 100
    for fiber in fiber_types
    if pd.notna(selected[fiber]) and selected[fiber] > 0
}

print(demand_dict)

{'Cotton': 0.009040882454764856, 'Polyester': 0.0006543149284704662}


In [6]:
import bw2data as bd
import bw2io as bi
import bw2calc as bc
PROJECT_NAME = "<Textile_SY14>"

bi.restore_project_directory(
    fp='/etc/data/ecoinvent-3.10-cutoff-bw25.tar.gz',
    project_name=PROJECT_NAME,
)
bd.projects.set_current(PROJECT_NAME)

Restoring project backup archive - this could take a few minutes...
Restored project: <Textile_SY14>
07:33:14+0000 [info     ] Applying automatic update: 4.0 database search directories FTS5
07:33:14+0000 [info     ] Reindexing database ecoinvent-3.10-biosphere
07:33:15+0000 [info     ] Reindexing database ecoinvent-3.10-cutoff


In [7]:
bd.databases

Databases dictionary with 2 object(s):
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff

In [8]:
db = bd.Database("ecoinvent-3.10-cutoff")

In [9]:
fibers = ["cotton", "polyester", "acrylic", "viscose", "silk", "wool", "nylon"]
results = {}

for fiber in fibers:
    matches = [act for act in db if fiber in act['name'].lower()]
    results[fiber] = matches
for fiber, acts in results.items():
    print(f"\n{fiber.upper()}:")
    for act in acts:
        print(f"  {act['name']} - {act['location']}")    


COTTON:
  market for cottonseed meal - GLO
  continuous dyeing, fibre, cotton - GLO
  textile production, cotton, air jet loom weaving - IN
  market for yarn, cotton - GLO
  yarn production, cotton, ring spinning, for knitting - GLO
  market for cottonseed oil, refined - GLO
  cottonseed oil refinery operation - RoW
  seed-cotton production, organic - RoW
  fibre production, cotton, organic, ginning - IN
  seed-cotton production, conventional - IN-GJ
  market for cottonseed oil, crude - GLO
  textile production, cotton, weaving - BD
  market for seed-cotton - GLO
  market for finishing, textile, knit cotton - GLO
  cottonseed, organic to generic market for energy feed - IN
  cottonseed production, for sowing - RoW
  cottonseed meal to generic market for protein feed - GLO
  fibre production, cotton, organic, ginning - RoW
  cottonseed, organic to generic market for energy feed - IN
  textile production, cotton, weaving - RoW
  fibre production, cotton, ginning - RoW
  batch dyeing, fi

In [10]:
def get_unique_node(name, location, unit, reference_product, database="ecoinvent-3.10-cutoff"):
    matches = [
        act for act in bd.Database(database)
        if act.get('name') == name
        and act.get('location') == location
        and act.get('unit') == unit
        and act.get('reference product') == reference_product
    ]
    
    if len(matches) == 0:
        raise ValueError(f"No match found for: {name} - {location} - {unit} - {reference_product}")
    elif len(matches) == 1:
        return matches[0]
    elif len(matches) > 1:
        print(f"⚠️ Multiple matches found for: {name} - {location} - {reference_product}")
        for i, act in enumerate(matches):
            print(f"\n--- Match {i + 1} ---")
            print(f"Filename: {act.get('filename')}")
            print(f"Comment: {act.get('comment', '')[:300]}")
        print("\n✅ Defaulting to first match.\n")
    
    return matches[0]


In [11]:
fiber_activities = {}

fiber_activities["acrylic"] = get_unique_node("acrylic acid production", "RER", "kilogram", "acrylic acid")
fiber_activities["cotton"] = get_unique_node("fibre production, cotton, ginning", "RoW", "kilogram", "fibre, cotton")
fiber_activities["polyester"] = get_unique_node("polyester fibre production, finished", "RoW", "kilogram", "fibre, polyester")
fiber_activities["viscose"] = get_unique_node("fibre production, viscose", "GLO", "kilogram", "fibre, viscose")
fiber_activities["silk"] = get_unique_node("market for fibre, silk, short", "GLO", "kilogram", "fibre, silk, short")
fiber_activities["nylon"] = get_unique_node("nylon 6-6 production", "RoW", "kilogram", "nylon 6-6")
fiber_activities["wool"] = get_unique_node("sheep production, for wool", "US", "kilogram", "sheep fleece in the grease")


In [12]:
for fiber, act in fiber_activities.items():
    print(f"\n🔍 Fiber: {fiber}")
    print(f"  Name: {act['name']}")
    print(f"  Location: {act['location']}")
    print(f"  Reference product: {act.get('reference product')}")
    print(f"  Unit: {act['unit']}")




🔍 Fiber: acrylic
  Name: acrylic acid production
  Location: RER
  Reference product: acrylic acid
  Unit: kilogram

🔍 Fiber: cotton
  Name: fibre production, cotton, ginning
  Location: RoW
  Reference product: fibre, cotton
  Unit: kilogram

🔍 Fiber: polyester
  Name: polyester fibre production, finished
  Location: RoW
  Reference product: fibre, polyester
  Unit: kilogram

🔍 Fiber: viscose
  Name: fibre production, viscose
  Location: GLO
  Reference product: fibre, viscose
  Unit: kilogram

🔍 Fiber: silk
  Name: market for fibre, silk, short
  Location: GLO
  Reference product: fibre, silk, short
  Unit: kilogram

🔍 Fiber: nylon
  Name: nylon 6-6 production
  Location: RoW
  Reference product: nylon 6-6
  Unit: kilogram

🔍 Fiber: wool
  Name: sheep production, for wool
  Location: US
  Reference product: sheep fleece in the grease
  Unit: kilogram


In [13]:
import numpy as np

method = [m for m in bd.methods if "IPCC 2013" in str(m) and "climate change" in str(m)][0]

# Step 2: List of activities
products = list(fiber_activities.values())  

# Step 3: Prepare results array
results = np.zeros((len(products), 1))

# Step 4: Loop over products and compute LCIA
for i, act in enumerate(products):
    try:
        lca = bc.LCA({act: 1}, method=method)
        lca.lci()
        lca.lcia()
        results[i, 0] = lca.score
    except Exception as e:
        print(f"Error for {act['name']}: {e}")
        results[i, 0] = np.nan 

for i, act in enumerate(products):
    print(f"{act['name']} → {results[i, 0]:.3f} kg CO₂-eq")


acrylic acid production → 2.268 kg CO₂-eq
fibre production, cotton, ginning → 2.014 kg CO₂-eq
polyester fibre production, finished → 4.459 kg CO₂-eq
fibre production, viscose → 3.078 kg CO₂-eq
market for fibre, silk, short → 0.010 kg CO₂-eq
nylon 6-6 production → 7.024 kg CO₂-eq
sheep production, for wool → 20.221 kg CO₂-eq
